Adapted from the following, but changed to handle multi-label
https://github.com/VanekPetr/flan-t5-text-classifier/blob/main/classifier/AutoModelForSeq2SeqLM/flan-t5-finetuning.py

# Login to Hugging Face

In [ ]:
!pip install -q transformers datasets sentencepiece accelerate evaluate peft bitsandbytes protobuf scikit-learn
from huggingface_hub import login
import os

login(token=os.getenv("HF_TOKEN"))

#from huggingface_hub import notebook_login
#notebook_login()
# Setup

In [1]:
import warnings
warnings.filterwarnings('ignore')

base_model_id = 'google/flan-t5-base'

In [2]:

seed=2024

# Training
num_train_epochs=4
batch_size = 8

#learning_rate = 5e-5
learning_rate=3e-4
#learning_rate = 1e-3

# Regularisation
dropout_rate = 0.1
#weight_decay=0.0001

# Evaluation
label_threshold=0.5

# Misc
results_output_dir = 'results'
logging_dir='logs'



hf_site_id = '2024-mcm-everitt-ryan'
dataset_id = f'{hf_site_id}/job-bias-synthetic-human-benchmark'
#dataset_id = f'{hf_site_id}/job-bias-synthetic-human-verified'
base_model_name = base_model_id.split('/')[-1]
model_id = f'{base_model_name}-job-bias-seq2seq-cls'
hub_model_id = f'{hf_site_id}/{model_id}'

In [3]:
!pip install -q transformers datasets sentencepiece accelerate evaluate hf_transfer huggingface_hub scikit-learn protobuf nltk

# Dataset

In [4]:
from datasets import load_dataset

dataset = load_dataset(dataset_id)
column_names = dataset['train'].column_names


text_col = 'text'
label_cols = [col for col in column_names if col.startswith('label_')]

labels = [label.replace("label_", "") for label in label_cols]

id2label = {idx: label for idx, label in enumerate(labels)}
label2id = {label: idx for idx, label in enumerate(labels)}

# Remove all columns apart from the two needed for multi-class classification
keep_columns = ['id', text_col] + label_cols
for split in ["train", "val", "test"]:
    dataset[split] = dataset[split].remove_columns(
        [col for col in dataset[split].column_names if col not in keep_columns])

for type in ['train','val','test']:
    dataset[type] = dataset[type].shuffle(seed=seed)#.select(range(10))

dataset

# Tokeniser

In [5]:
from transformers import AutoTokenizer
from huggingface_hub import HfFolder

tokenizer = AutoTokenizer.from_pretrained(base_model_id, token=HfFolder.get_token())
tokenizer

In [6]:
from datasets import concatenate_datasets
from transformers import Seq2SeqTrainingArguments


tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(
    lambda x: tokenizer(x["text"], truncation=True),
    batched=True,
    remove_columns=dataset['train'].column_names,
)
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# Prepare target sequences for T5
def create_target_sequence(example):
    labels = [key.replace('label_','') for key, value in example.items() if key.startswith('label_') and value]
    labels = ','.join(labels)
    labels = labels.strip()    
    return labels

# Add target sequence to the dataset
dataset = dataset.map(lambda x: {'labels': create_target_sequence(x)}, remove_columns=[col for col in dataset['train'].column_names if col.startswith('label_')])

# Tokenise targets
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(
    lambda x: tokenizer(x["labels"], truncation=True),
    batched=True,
    remove_columns=dataset['train'].column_names,
)
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

In [7]:
#tokenized_targets["input_ids"]

# Model

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoConfig

config = AutoConfig.from_pretrained(base_model_id, dropout_rate=dropout_rate)

model = AutoModelForSeq2SeqLM.from_pretrained(
    base_model_id,
    config=config
)
model

In [9]:
model.config

# Preprocessing/Evaluation functions

In [10]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, \
    classification_report
import nltk
from transformers import  DataCollatorForSeq2Seq, Seq2SeqTrainer
import numpy as np
from nltk import sent_tokenize
from typing import List, Tuple
from datasets import Dataset


def preprocess_function(sample: Dataset, padding: str = "max_length") -> dict:
    """Preprocess the dataset."""
    inputs = [item for item in sample["text"]]
    labels = [item for item in sample["labels"]]

    model_inputs = tokenizer(
        inputs, max_length=max_source_length, padding=padding, truncation=True
    )

    labels = tokenizer(
        text_target=labels, max_length=max_target_length, padding=padding, truncation=True
    )

    if padding == "max_length":
        labels["input_ids"] = [
            [(la if la != tokenizer.pad_token_id else -100) for la in label]
            for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def postprocess_text(labels: List[str], preds: List[str]) -> Tuple[List[str], List[str]]:
    """Helper function to postprocess text"""
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]
    return labels, preds


def compute_metrics(eval_predictions):
    
    y_hat, y = eval_predictions
    
    # Replace -100 in the labels .
    y = np.where(y != -100, y, tokenizer.pad_token_id)
    
    if isinstance(y_hat, tuple):
        y_hat = y_hat[0]
        
    #print(y)
    #print('--------------------')
    #print(y_hat)
    
    y_str = tokenizer.batch_decode(y, skip_special_tokens=True)
    y_hat_str = tokenizer.batch_decode(y_hat, skip_special_tokens=True)

    y_str, y_hat_str = postprocess_text( y_str, y_hat_str)
    
    #print('--------------------')
    #print(y_str)
    #print('--------------------')
    #print(y_hat_str)
    #print('--------------------')


    # Flatten the list of labels
    true_flat = [label.strip() for sublist in [t.split(',') for t in y_str] for label in sublist]
    pred_flat = [label.strip() for sublist in [p.split(',') for p in y_hat_str] for label in sublist]
    
    
    #print(true_flat)
    #print('--------------------')
    #print(pred_flat)

    # Convert to binary format for multi-label metrics
    #unique_labels = list(set(true_flat + pred_flat))  # This will include out-of-scope (not a label) predictions
    unique_labels = list(set(true_flat))
    
    # Remove the blank label (no bias)
    unique_labels = [label for label in unique_labels if label != '' and label is not None]
    
    y_true = [[1 if label in t else 0 for label in unique_labels] for t in y_str]
    y_pred = [[1 if label in p else 0 for label in unique_labels] for p in y_hat_str]

    #unique_labels = ['no_bias' if not label else label for label in unique_labels]

    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)

    f1_micro = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    f1_macro = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    f1_samples = f1_score(y_true=y_true, y_pred=y_pred, average='samples')
    f1_weighted = f1_score(y_true=y_true, y_pred=y_pred, average='weighted')

    precision_micro = precision_score(y_true=y_true, y_pred=y_pred, average='micro')
    recall_micro = recall_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc_micro = roc_auc_score(y_true=y_true, y_score=y_pred, average='micro')
    
    print(classification_report(y_true, y_pred, target_names=unique_labels))#, target_names=list(id2label.values())))

    
    #for i in range(len(y_true_str)):
    #    yt  = [num for num in y_true_str[i] if num != 0]
    #    yth  = [num for num in y_pred_str[i] if num != 0]
    #    print(f't: {yt}')
    #    print(f'p: {yth}')
    #    print('-----------------')
    #    print(y_true[i])
    #    print(y_pred[i])
    #    print('----------------------')
    
    metrics = {
        'accuracy': accuracy,
        f'f1_micro': f1_micro,
        f'f1_macro': f1_macro,
        f'f1_samples': f1_samples,
        f'f1_weighted': f1_weighted,
        f'precision_micro': precision_micro,
        f'recall_micro': recall_micro,
        f'roc_auc_micro': roc_auc_micro}
    return metrics


# Train

In [11]:
from transformers import TrainerCallback

args = Seq2SeqTrainingArguments(
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    learning_rate=learning_rate,
    output_dir=results_output_dir,
    #logging_dir=logging_dir,  # logging & evaluation strategies
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
   # weight_decay=weight_decay
    load_best_model_at_end=True,
    metric_for_best_model='loss',
    predict_with_generate=True,
    fp16=False,  # Overflows with fp16
    #report_to="tensorboard",
    #push_to_hub=True,
    #hub_strategy="every_save",
    #hub_model_id=REPOSITORY_ID,
    #hub_token=HfFolder.get_token(),
)

encoded_dataset = dataset.map(
    preprocess_function, batched=True, remove_columns=["text", "labels"]
)
print(f"Keys of tokenized dataset: {list(encoded_dataset['train'].features)}")

nltk.download("punkt")

label_pad_token_id = -100
data_collator = DataCollatorForSeq2Seq(
    tokenizer, model=model, label_pad_token_id=label_pad_token_id, pad_to_multiple_of=8
)

#early_stop = transformers.EarlyStoppingCallback(10, 1.15)
class PrintClassificationCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, logs=None, **kwargs):
        print("----------------------------------------------------------")




In [12]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    data_collator=data_collator,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["val"],
    compute_metrics=compute_metrics,
    callbacks=[PrintClassificationCallback]
)

model.config.use_cache = False  # Silence the warnings.

!nvidia-smi

trainer.train()

# Evaluate

In [13]:
test_results = trainer.evaluate(eval_dataset=encoded_dataset['test'])
test_results

In [ ]:

y_true = encoded_dataset['test']['labels']
predictions = trainer.predict(encoded_dataset['test'])
predictions = predictions.predictions

sigmoid = torch.nn.Sigmoid()
probs = sigmoid(torch.Tensor(predictions))
# use threshold to turn them into integer predictions
y_pred = np.zeros(probs.shape)
y_pred[np.where(probs >= label_threshold)] = 1

report = classification_report(y_true, y_pred, target_names=list(id2label.values()))
#print(report)

# Convert to Markdown
report_lines = report.split('\n')
markdown_classification_report = "\n".join([f"    {line}" for line in report_lines])
print(markdown_classification_report)

In [ ]:
import pandas as pd
df = pd.DataFrame(list(test_results.items()), columns=['Metric', 'Value'])
print(df.to_string(index=False))
import pandas as pd
df = pd.DataFrame(list(test_results.items()), columns=['Metric', 'Value'])
print(df.to_string(index=False))

In [15]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def classify_text(text, model, tokenizer, label_columns, device):
    input_text = f"classify: {text}"
    inputs = tokenizer(input_text, return_tensors='pt', padding=True, truncation=True).to(device)
    outputs = model.generate(**inputs)
    predicted_labels = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predicted_labels = [label.strip() for label in predicted_labels.split(',')]
    label_dict = {label: False for label in label_columns}
    for label in predicted_labels:
        if label in label_dict:
            label_dict[label] = True
    return label_dict

In [16]:
text = "Looking for a native English speaker"

classify_text(text, model, tokenizer, labels, device)

# Push to Hugging Face

In [18]:

from huggingface_hub import ModelCard, EvalResult, ModelCardData
import platform
import sys
import os

model.push_to_hub(repo_id=hub_model_id, token=HfFolder.get_token())
tokenizer.push_to_hub(repo_id=hub_model_id, token=HfFolder.get_token())

###### Update Model Card ######

eval_results = []
for k, v in test_results.items():
    eval_results.append(EvalResult(
        task_type='multi_label_classification',
        dataset_type='mix_human-eval_synthetic',
        dataset_name=dataset_id,
        metric_type=k.replace("eval_", "", 1),
        metric_value=v))

direct_use = """
    ```python
    from transformers import pipeline

    pipe = pipeline("text-classification", model="${hub_model_id}", return_all_scores=True)

    results = pipe("Join our dynamic and fast-paced team as a Junior Marketing Specialist. We seek a tech-savvy and energetic individual who thrives in a vibrant environment. Ideal candidates are digital natives with a fresh perspective, ready to adapt quickly to new trends. You should have recent experience in social media strategies and a strong understanding of current digital marketing tools. We're looking for someone with a youthful mindset, eager to bring innovative ideas to our young and ambitious team. If you're a recent graduate or early in your career, this opportunity is perfect for you!")
    print(results)
    ```
    >> [[
    {'label': 'age', 'score': 0.9883460402488708}, 
    {'label': 'disability', 'score': 0.00787709467113018}, 
    {'label': 'feminine', 'score': 0.007224376779049635}, 
    {'label': 'general', 'score': 0.09967829287052155}, 
    {'label': 'masculine', 'score': 0.0035264550242573023}, 
    {'label': 'racial', 'score': 0.014618005603551865}, 
    {'label': 'sexuality', 'score': 0.005568435415625572}
    ]]


    Classification Report:
    
    ${markdown_classification_report}
    """
direct_use = direct_use.replace('${hub_model_id}', hub_model_id, -1)
direct_use = direct_use.replace('${markdown_classification_report}', markdown_classification_report, -1)

card_data = ModelCardData(
    model_id=model_id,
    model_name=model_id,
    model_description="The model is a multi-label classifier designed to detect various types of bias within job descriptions.",
    base_model=base_model_id,
    language='en',
    license='apache-2.0',
    developers="Tristan Everitt and Paul Ryan",
    model_card_authors='See developers',
    model_card_contact='See developers',
    repo="https://gitlab.computing.dcu.ie/everitt2/2024-mcm-everitt-ryan",
    eval_results=eval_results,
    compute_infrastructure=f'{platform.system()} {platform.release()} {platform.processor()}',
    # hardware_requirements=f"CPUs: {psutil.cpu_count()}, Memory: {psutil.virtual_memory().total} bytes",
    software=f'Python {platform.python_version()}',
    hardware_type=platform.machine(),
    hours_used='N/A',
    cloud_provider='N/A',
    cloud_region='N/A',
    co2_emitted='N/A',
    datasets=[dataset_id],
    direct_use=direct_use
)

card = ModelCard.from_template(card_data)

card.push_to_hub(repo_id=hub_model_id, token=HfFolder.get_token())